# Decision Trees

## Imports

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

## Loading data

In [ ]:
data_path = '../data/dataset.csv'
data = pd.read_csv(data_path)

# Assuming the last column is the target variable
X = data.iloc[:, :-1]  # Features
y = data.iloc[:, -1]  # Target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Basic model

### Creation and training

In [ ]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

### Evaluation

In [ ]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

### Visualitzation

In [ ]:
plt.figure(figsize=(20,10))
plot_tree(clf, filled=True, feature_names=X.columns, class_names=True, max_depth=3)
plt.show()

## Custom model

### Params selection

Los parámetros elegidos son los siguientes:
- criterion: Define la métrica utilizada para medir la calidad de las divisiones. Puede ser el índice de Gini (que mide la impureza) o la entropía (basada en information gain).
- splitter: Determina la estrategia para seleccionar la división en cada nodo. Puede buscar la mejor división posible o seleccionar una aleatoria entre las mejores opciones.
- max_depth: Limita la profundidad máxima del árbol, lo que ayuda a controlar el sobreajuste. Si se deja como `None`, el árbol crecerá hasta que todas las hojas sean puras o contengan menos muestras que el mínimo permitido.
- min_samples_split: Es el número mínimo de muestras necesarias para dividir un nodo. Valores más altos evitan divisiones excesivas y reducen el riesgo de sobreajuste.
- min_samples_leaf: Es el número mínimo de muestras que debe tener una hoja. Esto asegura que las hojas no sean demasiado pequeñas.
- class_weight: Permite ajustar el peso de las clases para manejar datasets desbalanceados. La opción `'balanced'` ajusta automáticamente los pesos inversamente proporcionales a la frecuencia de las clases.
- min_impurity_decrease: Es el umbral mínimo de reducción de impureza requerido para realizar una división. Ayuda a evitar divisiones que no aporten suficiente valor.

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'class_weight': [None, 'balanced'],
    'min_impurity_decrease': [0.0, 0.01, 0.02]
}

### Creation and training

In [ ]:
grid_search = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid,
    cv=10,  # 10-fold cross-validation
    scoring='f1_macro',
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train, y_train)

print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

### Evaluation

In [ ]:
best_model = grid_search.best_estimator_

y_pred_best = best_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred_best)
print(f'Accuracy: {accuracy:.2f}')
print('Classification Report:')
print(classification_report(y_test, y_pred_best))
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred_best))

print('Most important features')
feature_importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

print(feature_importances.head(10))